<a href="https://colab.research.google.com/github/gyuchangShim/MJU_AI/blob/MASK-RCNN/Mask_R_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# detectron2 git clone 및 환경 설정
!python -m pip install pyyaml==5.1
import sys, os, distutils.core
# Note: This is a faster way to install detectron2 in Colab, but it does not include all functionalities (e.g. compiled operators).
# See https://detectron2.readthedocs.io/tutorials/install.html for full installation instructions
!git clone 'https://github.com/facebookresearch/detectron2'
dist = distutils.core.run_setup("./detectron2/setup.py")
!python -m pip install {' '.join([f"'{x}'" for x in dist.install_requires])}
sys.path.insert(0, os.path.abspath('./detectron2'))

In [ ]:
# torch, cuda, detectron2 version 확인
import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

In [3]:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, random
import matplotlib.pyplot as plt

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultTrainer, DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.data.datasets import register_coco_instances
from detectron2.evaluation import COCOEvaluator
from detectron2.data import build_detection_test_loader

In [ ]:
import os
import tensorflow as tf
from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

tf.test.is_gpu_available()
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

In [5]:
# Drive Mount
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import json

# 파일을 UTF-8 인코딩으로 읽어오기
with open(train_annotations_path, "r", encoding="utf-8") as f:
    data = json.load(f)

# COCO 데이터셋 등록
register_coco_instances("pole_train", {}, data["images"], data["annotations"])

# 데이터셋 메타데이터 로드
pole_metadata = MetadataCatalog.get("pole_train")


In [ ]:
dataset_dir = "/content/gdrive/MyDrive/pole_dataset/"
train_images_path = os.path.join(dataset_dir, "images")
train_annotations_path = os.path.join(dataset_dir, "annotations", "instances_default.json")
#val_images_path = os.path.join(dataset_dir, "val", "images")
#val_annotations_path = os.path.join(dataset_dir, "val", "annotations", "instances_val.json")

# COCO 데이터셋 등록
register_coco_instances("pole_train", {}, train_annotations_path, train_images_path)
#register_coco_instances("pole_val", {}, val_annotations_path, val_images_path)

# 데이터셋 메타데이터 로드
pole_metadata = MetadataCatalog.get("pole_train")

In [10]:
# 기본 설정
cfg = get_cfg()
# COCO 데이터 셋에 사전 학습된 MASK R-CNN 모델의 기본 설정 파일 가져오기
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
# 훈련 데이터 셋 지정
cfg.DATASETS.TRAIN = ("pole_train",)
# 모델 테스트 데이터 셋 지정
cfg.DATASETS.TEST = ()
# Data Loader에 필요한 worker 프로세스의 수 지정(많을 수록 빨라짐)
cfg.DATALOADER.NUM_WORKERS = 2
# COCO 데이터 셋에서 사전 학습된 모델의 가중치 가져오기(가중치 초기화)
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")  # 사전 학습된 가중치 사용
# 한 Batch에 포함되는 이미지 개수 지정
cfg.SOLVER.IMS_PER_BATCH = 2
# 학습의 기본 학습률 설정(모델의 학습 속도를 결정)
cfg.SOLVER.BASE_LR = 0.00025
# 학습의 반복 횟수 지정
cfg.SOLVER.MAX_ITER = 100
# 각 이미지 당 ROI에서 추출할 샘플의 개수 지정(학습 중 사용)
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 512
# 모델이 예측할 클래스 개수 지정
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 5

In [ ]:
# 이전 프로세스 확인
import psutil

for proc in psutil.process_iter():
    try:
        process_name = proc.name()
        if "python" in process_name and "colab" in process_name:
            print("이전 프로세스 발견:", process_name)
    except (psutil.NoSuchProcess, psutil.AccessDenied, psutil.ZombieProcess):
        pass

print("모든 프로세스 검색 완료.")

In [ ]:
# CUDA 디버깅
#os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
#os.environ["CUDA_VISIBLE_DEVICES"] = "0"
#os.environ["TORCH_USE_CUDA_DSA"] = '1'

# Trainer 생성
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

In [ ]:
# Tensorboard를 활용한 결과 확인
%load_ext tensorboard
%tensorboard --logdir output